In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "./DeepSeek-R1-Distill-Qwen-1.5B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 02-13 09:39:40 __init__.py:190] Automatically detected platform cuda.
==((====))==  Unsloth 2025.2.5: Fast Qwen2 patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.546 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
./DeepSeek-R1-Distill-Qwen-1.5B does not have a padding token! Will use pad_token = <|vision_pad|>.


In [2]:
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.
Please answer the following medical question.

### Question:
{}

### Response:
<think>{}"""

In [3]:
question = "一个患有急性阑尾炎的病人已经发病5天，腹痛稍有减轻但仍然发热，在体检时发现右下腹有压痛的包块，此时应如何处理？"


FastLanguageModel.for_inference(model)
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])


<think>
好，我现在要处理一个关于急性阑尾炎患者的临床问题。首先，病人已经发病5天，腹痛有轻微减轻，但发热仍然存在。体检时发现右下腹有压痛包块。我需要考虑处理措施，可能包括药物、手术或治疗。

首先，急性阑尾炎通常由感染性或非感染性的因素引起，比如乙型脑炎。患者发热可能与感染有关，但也可能与炎症反应有关。但腹痛和包块提示有压痛，这可能与感染有关，也可能与炎症反应有关。

接下来，我需要评估患者的体征状况。腹痛和发热，但没有明显的低烧，所以不能确定是否感染。包块可能影响排液，影响治疗效果，所以需要考虑药物治疗。

首先，考虑抗生素治疗。急性阑尾炎可能有细菌感染，抗生素治疗可能有助于缓解症状，但可能引起炎症反应。但患者发热，可能需要同时考虑抗炎药物，比如β受体阻滞剂或抗生素联合β受体阻滞剂。

同时，需要考虑包块处理。包块可能影响排液，需要药物治疗，如β受体阻滞剂，或者手术，如开腹或镜下手术，如清肠手术或阑尾切除术，如果包块太大或形状不规则。

此外，患者是否需要进一步的治疗，比如抗生素联合β受体阻滞剂，或者手术。如果包块较大或形状不规则，可能需要手术。

最后，评估患者的预后。药物治疗可能缓解症状，但可能引起炎症反应。手术可能需要 longer recovery，但可能更准确治疗。需要根据患者的具体情况和医生建议来决定。

总结一下，处理步骤可能包括抗生素治疗，同时考虑包块处理，可能需要药物或手术。
</think>

对于一个患有急性阑尾炎的病人，患者已经发病5天，腹痛轻微减轻但仍有发热，体检时右下腹有压痛包块。应采取药物治疗，同时考虑包块处理，可能需要抗生素治疗和β受体阻滞剂。建议在医生指导下进行药物治疗，同时考虑手术，如清肠手术或阑尾切除术，以确保治疗的准确性。<｜end▁of▁sentence｜>


In [4]:
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.
Please answer the following medical question.

### Question:
{}

### Response:
<think>
{}
</think>
{}"""

In [5]:
EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN


def formatting_prompts_func(examples):
    inputs = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []
    for input, cot, output in zip(inputs, cots, outputs):
        text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }

In [7]:
from datasets import load_dataset
dataset = load_dataset("./medical-o1-reasoning-SFT", 'zh', split = "train[0:500]", trust_remote_code=True)
print(dataset.column_names)

Generating train split: 0 examples [00:00, ? examples/s]

['Question', 'Complex_CoT', 'Response']


In [8]:
dataset = dataset.map(formatting_prompts_func, batched = True)
dataset["text"][0]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

'Below is an instruction that describes a task, paired with an input that provides further context.\nWrite a response that appropriately completes the request.\nBefore answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.\n\n### Instruction:\nYou are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.\nPlease answer the following medical question.\n\n### Question:\n根据描述，一个1岁的孩子在夏季头皮出现多处小结节，长期不愈合，且现在疮大如梅，溃破流脓，口不收敛，头皮下有空洞，患处皮肤增厚。这种病症在中医中诊断为什么病？\n\n### Response:\n<think>\n这个小孩子在夏天头皮上长了些小结节，一直都没好，后来变成了脓包，流了好多脓。想想夏天那么热，可能和湿热有关。才一岁的小孩，免疫力本来就不强，夏天的湿热没准就侵袭了身体。\n\n用中医的角度来看，出现小结节、再加上长期不愈合，这些症状让我想到了头疮。小孩子最容易得这些皮肤病，主要因为湿热在体表郁结。\n\n但再看看，头皮下还有空洞，这可能不止是简单的头疮。看起来病情挺严重的，也许是脓肿没治好。这样的情况中医中有时候叫做禿疮或者湿疮，也可能是另一种情况。\n\n等一下，头皮上的空洞和皮肤增厚更像是疾病已经深入到头皮下，这是不是说明有可能是流注或瘰疬？这些名字常描述头部或颈部的严重感染，特别是有化脓不愈合，又形成通道或空洞的情况。\n\n仔细想想，我怎么感觉这些症状更贴近瘰疬的表现？尤其考虑到孩子的年纪和夏天发生的季节性因素，湿热可能是主因，但可能也有火毒或者痰湿造成的滞留。\n\n回到基本

In [9]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.2.5 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [10]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        # num_train_epochs = 1, # For longer training runs!
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Map (num_proc=2):   0%|          | 0/500 [00:00<?, ? examples/s]

In [11]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 500 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 18,464,768


Step,Training Loss
1,2.913300
2,3.278500
3,3.003800
4,3.035400
5,2.532500
6,2.769800
7,3.066000
8,2.933600
9,2.928000
10,2.911500


In [12]:
print(question)

一个患有急性阑尾炎的病人已经发病5天，腹痛稍有减轻但仍然发热，在体检时发现右下腹有压痛的包块，此时应如何处理？


In [14]:
FastLanguageModel.for_inference(model)  # Unsloth has 2x faster inference!
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])


<think>
嗯，病人已经发病5天了，腹痛稍微减轻了一点，但还是有一点发热。现在，我们看到右下腹有一个压痛包块，这让我有点担心。包块可能影响了我们的生命，所以需要特别小心。

首先，压痛包块可能是因为下腹部的炎症或者其他原因引起的。病人已经得了急性阑尾炎，所以下腹部的炎症可能会进一步恶化。

接下来，我想了解一下，这种情况下，我们通常会考虑哪些处理方法呢？比如，先尝试一下冷敷，这样可以减轻疼痛和发热，同时还能帮助下腹部的炎症减轻。如果冷敷还没有效，我们还可以考虑使用止痛药，比如西替利嗪，这样可以减轻疼痛和发热。

不过，如果冷敷和止痛药都没能解决，那我们就得考虑更紧急的措施了。比如，如果病人还存在感染性包块，可能需要进行手术。不过，如果包块已经比较小，或者疼痛已经缓解，可能不需要那么紧急。

另外，病人有急性阑尾炎，所以下腹部的炎症可能会更严重。所以，我们可能需要考虑使用抗生素来帮助控制炎症。但抗生素可能会让症状更严重，所以需要谨慎选择。

还有一点，病人有发热，这种情况通常需要更多的液体帮助，比如抗休克液，来帮助维持正常代谢。如果病人不能喝进去液体，或者如果他有其他并发症，比如感染性包块，可能需要更紧急的措施。

综合来看，我们首先应该尝试冷敷和止痛药，看看有没有缓解症状的效果。如果效果还不明显，再考虑手术，但如果症状已经缓解了，可能不需要那么紧急。

总之，根据这些考虑，我们可能会先试试冷敷，然后考虑止痛药和抗生素，最后再考虑手术。
</think>
根据病人的情况，右下腹压痛包块已经存在，结合急性阑尾炎的特征，建议进行冷敷处理。首先，建议使用冷敷，可以减轻疼痛和发热。如果冷敷效果不佳，可以尝试止痛药如西替利嗪，帮助减轻疼痛和发热。如果症状依然没有改善，可以考虑使用抗生素来控制炎症。如果病人有感染性包块，可能需要进行手术。但根据病人目前的症状和症状缓解情况，初步考虑冷敷和止痛药作为初步措施。<｜end▁of▁sentence｜>
